### Import modules

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# General
import os
import random

# Data processing, storage
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Image processing
import cv2 # opencv-python
from PIL import Image

# Data plotting
import matplotlib.pyplot as plt

# Machine learning
import tensorflow as tf
from sklearn import metrics

### Check input data availability

In [ ]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname, "contains", len(filenames), "files.")
    #for filename in filenames:
    #    print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Set parameters

In [ ]:
basedatadir = '/kaggle/input/p7eglmligltilesshiftrotate/P7-EGL-ML-IGL-tiles-shift-rotate'

train_data_dir = basedatadir + '/train'
test_data_dir = basedatadir + '/test'

# Calculation of max batch size-
# Max batch size= available GPU memory bytes / 4 / (size of tensors + trainable parameters)
# size of tensors = batch_size*100*100 *4 if using 64 bit integers

batch_size = 256
test_batch_size = 256
img_height = 300
img_width = 300
mean = 157.1
std = 64.6
AUTOTUNE = tf.data.experimental.AUTOTUNE

### Create FileList datasets
tf.data.Dataset.list_files() creates a dataset from a directory list of files using a matching pattern.

In [ ]:
train_list_ds = tf.data.Dataset.list_files(str(train_data_dir + '/*/*'), shuffle=False)
# get the count of image files in the train directory
train_image_count=0
for dir1 in os.listdir(train_data_dir):
    for files in os.listdir(os.path.join(train_data_dir, dir1)):
        train_image_count+=1
train_list_ds = train_list_ds.shuffle(train_image_count, reshuffle_each_iteration=False)
print(train_image_count)

In [ ]:
# Print some filenames
for x in train_list_ds.take(5):
    print(x.numpy().decode('utf-8'))

In [ ]:
test_list_ds = tf.data.Dataset.list_files(str(test_data_dir + '/*/*'), shuffle=False)
# get the count of image files in the train directory
test_image_count=0
for dir1 in os.listdir(test_data_dir):
    for files in os.listdir(os.path.join(test_data_dir, dir1)):
        test_image_count+=1
test_list_ds = test_list_ds.shuffle(test_image_count, reshuffle_each_iteration=False)
print(test_image_count)

In [ ]:
# Print some filenames
for x in test_list_ds.take(5):
    print(x.numpy().decode('utf-8'))

### Function to create list of files from tensorflow dataset

In [ ]:
# Use this function to list files from a filelist dataset, such as test_list_ds.
# This may be useful later
list_files_from_tfdataset = lambda tfd: [f.numpy().decode('utf-8') for f in tfd]

In [ ]:
#l = list_files_from_tfdataset(test_list_ds)
#print(len(l))
#del(l)
#test_ds_filepaths = test_list_ds.take(test_image_count)

### Create class labels from the directory name

In [ ]:
class_names = np.array(sorted([dir1 for dir1 in os.listdir(train_data_dir)]))
class_names

### Split the dataset into train and val (Note that test dataset is already kept separate)
The validation dataset is 30% of the total dataset, and train dataset is 70% of the entire dataset.

In [ ]:
# Create training and validation datasets
val_size = int(train_image_count * 0.3)
train_ds = train_list_ds.skip(val_size)
val_ds = train_list_ds.take(val_size)
print("Training images:", train_image_count-val_size, "\nValidation images:", val_size)

In [ ]:
# Different way of creatint training and validation datasets
# This is useful to select only small subset of the full data
#val_size = int(train_image_count * 0.01)
#train_size = int(train_image_count * 0.03)
#train_ds = train_list_ds.take(train_size)
#remaining_list_ds = train_list_ds.skip(train_size)
#val_ds = remaining_list_ds.take(val_size)
#print("Training images:", train_size, "\nValidation images:", val_size)

In [ ]:
# Create test dataset
test_ds = test_list_ds.take(test_image_count)

In [ ]:
# Print some filenames
print("Files from train_ds")
for x in train_ds.take(3):
    print(x.numpy().decode('utf-8'))
print("Files from val_ds")
for x in val_ds.take(3):
    print(x.numpy().decode('utf-8'))
print("Files from test_ds")
for x in test_ds.take(3):
    print(x.numpy().decode('utf-8'))

### Create input pipeline components for a single training/validation example,<br>where a pair of tensors represents the image and its corresponding label.

In [ ]:
#To process the label
def get_label(filepath):
    # convert the path to a list of path components separated by sep
    parts = tf.strings.split(filepath, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.argmax(tf.cast(one_hot, tf.int32))
    
    #label = tf.strings.split(filepath, sep='/')
    #label = tf.strings.split(label[-1], sep='.')

# To process the image
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    #img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.decode_png(img, channels=1)
    # resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

# To create the single training of validation example with image and its corresponding label
def process_path(filepath):
    label = get_label(filepath)
    # load the raw data from the file as a string
    img = tf.io.read_file(filepath)
    img = decode_img(img)
    return img, label

### Create tf dataset of tensor by mapping the above functions to the filelist datasets<br>
Set the AUTOTUNE. This will help to delegate the decision on the level of parallelism<br>to use to the tf.data at runtime to optimize the CPU/GPU utilization.

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
# Test if the datasets are prepared correctly
for images, labels in train_ds.take(1):
    print(images.shape, labels.shape)
    img = images[0].numpy()
    print(img.min(), img.max(), img.dtype, labels.numpy())
    img = (img-5)/2 # Testing mathematical operations on tensor
    print(img.min(), img.max(), img.dtype, labels.numpy())

### Set augmentation and normalization functions

In [ ]:
def augment(image, label):
    img = tf.image.rot90(image)
    img = tf.image.flip_left_right(img)
    #img = tf.clip_by_value(img, 0.0, 1.0)
    return img, label

def standardize_per_image(image, label):
    img = tf.image.per_image_standardization(image)
    return img, label

def standardize_dataset(image, label):
    # NOTE: mean and std have to be defined globally
    # Not checking that mean and std are > 0 to avoid time lag. So be careful with the values of mean and std.
    img = (image - mean)/std
    return img, label

### Configure data source for Performance
To configure the data source for performance, use prefetching. Prefetching in tf.data allows the preprocessing of the data and model execution of a training step to overlap. While the model is executing a training step 100, the input pipeline is reading the data for step 101.

In [ ]:
def configure_for_performance(ds, cache_filename=''):
    if cache_filename == '':
        ds = ds.cache()
    else:
        ds = ds.cache(cache_filename)
    ds = ds.shuffle(buffer_size=1000)
    #ds = ds.map(augment, num_parallel_calls=AUTOTUNE) # No need if pre-augmented images.
    #ds = ds.map(standardize_per_image, num_parallel_calls=AUTOTUNE) # Use either of the two standardizations.
    ds = ds.map(standardize_dataset, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds, cache_filename='')#, '/home/harsh/tensorflow_cache/train_v5.train_ds')
val_ds = configure_for_performance(val_ds, cache_filename='')#, '/home/harsh/tensorflow_cache/train_v5.val_ds')

In [ ]:
def configure_for_performance_forTestDataset(ds, cache_filename=''):
    if cache_filename == '':
        ds = ds.cache()
    else:
        ds = ds.cache(cache_filename)
    #ds = ds.shuffle(buffer_size=1000)
    #ds = ds.map(standardize_per_image, num_parallel_calls=AUTOTUNE) # Use either of the two standardizations.
    ds = ds.map(standardize_dataset, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(test_batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

test_ds = configure_for_performance_forTestDataset(test_ds, cache_filename='')#, '/home/harsh/tensorflow_cache/train_v5.test_ds')

In [ ]:
# Test that the pixels values are actually standardized and the images are properly stored.
for images, labels in train_ds.take(1):
    print(images.shape, labels.shape)
    img = images[0].numpy()
    print(img.min(), img.max(), img.dtype, labels[0].numpy())
    plt.imshow(img, cmap=plt.get_cmap("gray"))

In [ ]:
# Check the test dataset. This dataset is not shuffled.
# So you should see same values printed if you run it again.
for images, labels in test_ds.take(1):
    print(images.shape, labels.shape)
    #img = images[0].numpy()
    print(images[0].numpy().min(), images[0].numpy().max())
    print(images[100].numpy().min(), images[1].numpy().max())

In [ ]:
# Check if the file names can be matched with output of the test dataset
test_ds_filelist = list_files_from_tfdataset(test_list_ds) # Create test dataset file list.

In [ ]:
# Choose an image and it's filename from the dataset.
# Plot the tensor image as well as image given by the filename to see if they match.
for images, labels in test_ds.take(1):
    print(images.shape, labels.shape)
    #plt.imshow(images[2].numpy()[:,:], cmap=plt.get_cmap("gray"))
    print(images[2].numpy().shape, labels[2].numpy().shape)
    print("Class label:", labels[2].numpy())
    print("Class name:", class_names[labels[2].numpy()])
    print("File name:", test_ds_filelist[2])

f = plt.figure()

f.add_subplot(1,2, 1)
test_img = cv2.imread(test_ds_filelist[2], 0)
plt.imshow(images[2].numpy()[:,:], cmap=plt.get_cmap("gray"))

f.add_subplot(1,2, 2)
plt.imshow(test_img, cmap=plt.get_cmap("gray"))

plt.show(block=True)


In [ ]:
# Check if a batch of images from test dataset can be printed along with classnames and filenames
plt.figure(figsize=(35, 35))
cnt = 0
for images, labels in train_ds.take(1):
    #print(images.shape)
    for i in range(batch_size):
        ax = plt.subplot(int(np.sqrt(batch_size)), int(np.sqrt(batch_size)), i + 1)
        #ax = plt.subplot(16, 16, i + 1)
        #print(images[i].shape)
        plt.imshow(images[i].numpy(), cmap=plt.get_cmap("gray"))
        plt.title(class_names[labels[i]])
        #plt.title(class_names[labels[i]]+"\n"+test_ds_filelist[i+cnt*batch_size].split("/")[-1].replace(".png",""))
        plt.axis("off")
    cnt += 1